In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ai agent trainning/train/sft')

Mounted at /content/drive


In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
import yaml

with open('../configs/base.yaml', 'r') as file:
    configs = yaml.safe_load(file)

with open('../configs/sft.yaml', 'r') as file:
    configs.update(yaml.safe_load(file))

# Config model

In [ ]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
Standard import failed for UnslothNashMDTrainer: No module named 'UnslothNashMDTrainer'. Using tempfile instead!


In [ ]:
model_repo = configs['base_model']

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_repo,
    max_seq_length = configs['model']['max_seq_length'],
    dtype = configs['model']['dtype'],
    load_in_4bit = configs['model']['load_in_4bit'],
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = configs['model']['r'], # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules =configs['model']['target_modules'],
    lora_alpha = configs['model']['lora_alpha'],
    lora_dropout = configs['model']['lora_dropout'], # Supports any, but = 0 is optimized
    bias = configs['model']['bias'],    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = configs['model']['use_gradient_checkpointing'], # True or "unsloth" for very long context
    random_state = configs['model']['random_state'],
    use_rslora = configs['model']['use_rslora'],  # We support rank stabilized LoRA
    loftq_config = configs['model']['loftq_config'], # And LoftQ
)

Unsloth 2025.4.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Preprocess input text

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = configs['chat_template'],
)

def formatting_prompts_func(examples):
    convos = examples["conversation"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

In [ ]:
from datasets import load_dataset
data_path = configs['data_path']
dataset = load_dataset(data_path, split = "train")

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset[5]["conversation"]

[{'content': 'Tôi F 38 có thể trạng tốt, thường xuyên tập thể dục (tập ba môn phối hợp) nhưng bị đau lưng vì nhiều lý do khác nhau trong suốt cuộc đời, giờ đây tôi thường xuyên thức dậy với tình trạng lưng dưới trầm trọng, đau hông mà không rõ lý do. hôm nay cơn đau gần như khiến tôi nghẹt thở khi tôi cử động. Đó là một cơn đau âm ỉ khi tôi vừa nằm xuống nhưng ngay khi tôi thực hiện bất kỳ cử động nào, tôi lại cảm thấy đau nhói và đôi khi lan xuống chân.',
  'role': 'user'},
 {'content': 'Xin chào, Từ lịch sử, có vẻ như bạn có thể đang gặp phải những thay đổi thoái hóa ở cột sống lưng dưới, gây ra áp lực thần kinh bị chèn ép. Cũng có thể bị nhuyễn xương hoặc loãng xương. Chụp X-quang vùng thắt lưng cùng để điều trị viêm xương khớp. Vật lý trị liệu như các bài tập duỗi lưng sẽ rất hữu ích. Tiêm B1, B6, B!2 hoặc thuốc. Uống bổ sung canxi, vitamin A và D. Được rồi và bảo trọng nhé.',
  'role': 'assistant'}]

In [ ]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTôi F 38 có thể trạng tốt, thường xuyên tập thể dục (tập ba môn phối hợp) nhưng bị đau lưng vì nhiều lý do khác nhau trong suốt cuộc đời, giờ đây tôi thường xuyên thức dậy với tình trạng lưng dưới trầm trọng, đau hông mà không rõ lý do. hôm nay cơn đau gần như khiến tôi nghẹt thở khi tôi cử động. Đó là một cơn đau âm ỉ khi tôi vừa nằm xuống nhưng ngay khi tôi thực hiện bất kỳ cử động nào, tôi lại cảm thấy đau nhói và đôi khi lan xuống chân.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nXin chào, Từ lịch sử, có vẻ như bạn có thể đang gặp phải những thay đổi thoái hóa ở cột sống lưng dưới, gây ra áp lực thần kinh bị chèn ép. Cũng có thể bị nhuyễn xương hoặc loãng xương. Chụp X-quang vùng thắt lưng cùng để điều trị viêm xương khớp. Vật lý trị liệu như các bài tập duỗi lưng sẽ rất hữu ích. Tiêm B

In [ ]:
configs['trainning']['learning_rate']

'2e-4'

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = configs['model']['max_seq_length'],
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = configs['trainning']['per_device_train_batch_size'],
        gradient_accumulation_steps = configs['trainning']['gradient_accumulation_steps'],
        warmup_steps = configs['trainning']['warmup_steps'],
        # num_train_epochs = configs['trainning']['num_train_epochs'],
        max_steps = configs['trainning']['max_steps'],
        learning_rate = configs['trainning']['learning_rate'],
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = configs['trainning']['logging_steps'],
        optim = configs['trainning']['optim'],
        weight_decay = configs['trainning']['weight_decay'],
        lr_scheduler_type = configs['trainning']['lr_scheduler_type'],
        seed = configs['trainning']['seed'],
        output_dir = configs['trainning']['output_dir'],
        report_to = configs['trainning']['report_to'],
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTôi F 38 có thể trạng tốt, thường xuyên tập thể dục (tập ba môn phối hợp) nhưng bị đau lưng vì nhiều lý do khác nhau trong suốt cuộc đời, giờ đây tôi thường xuyên thức dậy với tình trạng lưng dưới trầm trọng, đau hông mà không rõ lý do. hôm nay cơn đau gần như khiến tôi nghẹt thở khi tôi cử động. Đó là một cơn đau âm ỉ khi tôi vừa nằm xuống nhưng ngay khi tôi thực hiện bất kỳ cử động nào, tôi lại cảm thấy đau nhói và đôi khi lan xuống chân.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nXin chào, Từ lịch sử, có vẻ như bạn có thể đang gặp phải những thay đổi thoái hóa ở cột sống lưng dưới, gây ra áp lực thần kinh bị chèn ép. Cũng có thể bị nhuyễn xương hoặc loãng xương. Chụp X-quang vùng thắt lưng cùng để điều trị viêm xương khớp. Vật lý trị liệu như các bài tập duỗi lưng sẽ rấ

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]]).strip()

'Xin chào, Từ lịch sử, có vẻ như bạn có thể đang gặp phải những thay đổi thoái hóa ở cột sống lưng dưới, gây ra áp lực thần kinh bị chèn ép. Cũng có thể bị nhuyễn xương hoặc loãng xương. Chụp X-quang vùng thắt lưng cùng để điều trị viêm xương khớp. Vật lý trị liệu như các bài tập duỗi lưng sẽ rất hữu ích. Tiêm B1, B6, B!2 hoặc thuốc. Uống bổ sung canxi, vitamin A và D. Được rồi và bảo trọng nhé.<|eot_id|>'

# Train

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.441 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 111,701 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.617200
2,2.641800
3,2.467200
4,2.859400
5,2.549600
6,2.427600
7,2.499200
8,2.570400
9,2.204500
10,2.546200


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

326.8626 seconds used for training.
5.45 minutes used for training.
Peak reserved memory = 4.148 GB.
Peak reserved memory for training = 0.707 GB.
Peak reserved memory % of max memory = 28.139 %.
Peak reserved memory for training % of max memory = 4.796 %.


In [ ]:
model.save_pretrained(configs['pretrain_model'])  # Local saving
tokenizer.save_pretrained(configs['pretrain_model'])
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('Llama-3.2-3B-Instruct/tokenizer_config.json',
 'Llama-3.2-3B-Instruct/special_tokens_map.json',
 'Llama-3.2-3B-Instruct/tokenizer.json')